In [1]:
from bs4 import BeautifulSoup
import urllib.request
import ssl
import pandas as pd
import os

def GetKospi200():     
    stockDic = dict()
    lastPageNum = 0

    # 마지막 페이지 찾기
    base_url = "http://finance.naver.com/sise/entryJongmok.nhn?&page="
    target_url = base_url + str(1)
    context = ssl._create_unverified_context()
    # soup = BeautifulSoup(urllib.request.urlopen(target_url).read(), "lxml")
    soup = BeautifulSoup(urllib.request.urlopen(target_url, context=context).read().decode('euc-kr', 'ignore'), "lxml")
    for item in soup.find_all('td'):
        if item.has_attr('class') and 'pgRR' in item['class']:
            lastPageNum = int(str(item.a['href']).replace('/sise/entryJongmok.nhn?&page=', ''))

    for i in range(1, lastPageNum+1):
        target_url = base_url + str(i)
        # soup = BeautifulSoup(urllib.request.urlopen(target_url).read(), "lxml")
        soup = BeautifulSoup(urllib.request.urlopen(target_url, context=context).read().decode('euc-kr', 'ignore'), "lxml")
        postNoList = soup.find_all('a')


        # 종목코드와 종목명 담기
        for item in postNoList:
            if item.has_attr('target') and '_parent' in item['target'] and item.has_attr('href'):
                if str(item['href']).startswith('/item/main.nhn?code='):
                    stockDic[str(item['href']).replace('/item/main.nhn?code=', '')] = item.text
    return stockDic

def getKospiIndex():
    outFileName = "data/kospi.csv"
    i = 1
    last_date = ""
    cols = ["date", "index", "diff", "diff_ratio", "amount", "tot_price"]
    if os.path.exists(outFileName):
        if "date" in pd.read_csv(outFileName).columns:
            df = pd.read_csv(outFileName)
        else:
            df = pd.read_csv(outFileName, header=None, names=cols) 
    while True:        
        url = "https://finance.naver.com/sise/sise_index_day.nhn?code=KOSPI&page=" + str(i)
        print(url)            
        table = pd.read_html(url)
        table = table[0].dropna() 
        table.columns = cols
        if last_date == table["date"].max():            
            break
        else:
            last_date = table["date"].max()
            if len(table[~table["date"].isin(df["date"].tolist())]) == 0:
                break
            
            df = df.append(table[~table["date"].isin(df["date"].tolist())]) 
            df.sort_values(by="date").to_csv(outFileName, index=False)
            i += 1

In [2]:
import pandas as pd

def GetPriceData(item):    
    
    # print(item)
    code = item[0]
    name = item[1]
                
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    url = "http://finance.naver.com/item/sise_day.nhn?code=" + code
    page = "&page="
    idx = 1
    
    directory = 'data/'
    outputFileName = directory + code + '.csv'
    bFileExist = False
    cols = ['date', 'code', 'start', 'close', 'min', 'max', 'amount']
    if os.path.exists(outputFileName):
        if 'date' not in pd.read_csv(outputFileName).columns:
            df = pd.read_csv(outputFileName, header=None, names=cols)
            df.sort_values(by="date").to_csv(outputFileName, index=False)
        df = pd.read_csv(outputFileName)
        bFileExist = True
        
    bFlag = True
    with open(outputFileName, 'a') as f:
        while bFlag == True:
            fullAddr = url + page + str(idx)

            source_code = requests.get(fullAddr, headers = headers)
            if source_code is None:
                bFlag = False
                return

            soup = BeautifulSoup(source_code.text,"lxml")
            if soup.find('td', class_='on').find('a').text != str(idx):
                bFlag = False
                return

            for tr in filter(lambda x:x.get("onmouseout") is not None, soup.find_all("tr")):
                if tr.find("span",class_ = "tah p10 gray03") is None:
                    # 가격데이터가 없으면 False로 빠져나옴
                    bFlag = False
                    break

                else:
                    tDate = tr.find("span",class_ = "tah p10 gray03").text
                    cPrice = tr.find_all("span",class_ = "tah p11")
                    sIdx = 1

                    if len(cPrice) != 5 :
                        sIdx = 2

                    dt = tDate.replace("." ,"-")
                    
                    if bFileExist and (df["date"] == dt).sum() > 0:                        
#                         print(dt, " 가격존재 ")
                        f.close()
                        bFlag = False
                        break

                    pClose = float(cPrice[0].text.replace("," ,""))
                    pStart = float(cPrice[sIdx].text.replace("," ,""))
                    sIdx += 1
                    pMax   = float(cPrice[sIdx].text.replace("," ,""))
                    sIdx += 1
                    pMin   = float(cPrice[sIdx].text.replace("," ,""))
                    sIdx += 1
                    amount = float(cPrice[sIdx].text.replace("," ,""))

                    stockList = []
                    stockList.append(dt)
                    stockList.append(code)
                    stockList.append(pStart)
                    stockList.append(pClose)
                    stockList.append(pMin)
                    stockList.append(pMax)
                    stockList.append(amount)                
                    f.writelines(",".join([str(x) for x in stockList]) + '\n') 
            idx += 1
    f.close()
    

In [3]:
from bs4 import BeautifulSoup
import urllib.request
import requests 
import os

def getFinanceInfo(code=None):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    # 투자정보
    # totalCnt = []  # 상장주식수
    # forignerHaveLimit = []  # 외국인한도보유주식수
    # forignerHaveCnt = []  # 외국인보유주식수
    # max52week = []  # 52주 최고
    # min52week = []  # 52주 최저
    # per = []
    # eps = []
    # per_eps_date = []
    # estimate_per = []  # 추정 PER
    # estimate_eps = []  # 추정 EPS
    # pbr = []
    # bps = []
    # pbr_bps_date = []
    # dvr = []  # 배당수익
    financeFileName = 'data/stock_finance_data.csv'
    cols = ["date",
             "code", 
             "totalCnt", 
             "forignerHaveLimit",
             "forignerHaveCnt",
             "min52week",
             "max52week",
             "per",
             "eps",
             "per_eps_date",
             "estimate_per",
             "estimate_eps",
             "pbr",
             "bps",
             "pbr_bps_date",
             "dvr"]
    if os.path.exists(financeFileName):
        if 'date' not in pd.read_csv(financeFileName).columns:            
            df = pd.read_csv(financeFileName, header=None, names=cols, dtype=str)            
            df.sort_values(by="date").to_csv(financeFileName, index=False)            
        df = pd.read_csv(financeFileName, dtype=str)
    
    url = "https://finance.naver.com/item/main.nhn?code=" + code
    source_code = requests.get(url, headers = headers)
    if source_code is None:
        pass
    else:
        soup = BeautifulSoup(source_code.text, "lxml")
        date = soup.find(id='time').find_all('em')[0].text
        
        # 이미 존재하는 데이터면 skip
        if ((df["date"] == date) & (df["code"] == code)).sum() > 0:
            return
        
        totalCnt = soup.find(id='tab_con1').find_all('em')[2].text.replace(',', '').replace('N/A', '0')
        forignerHaveLimit = soup.find(id='tab_con1').find_all('em')[5].text.replace(',', '').replace('N/A', '0')
        forignerHaveCnt = soup.find(id='tab_con1').find_all('em')[6].text.replace(',', '').replace('N/A', '0')
        max52week = soup.find(id='tab_con1').find_all('em')[10].text.replace(',', '').replace('N/A', '0')
        min52week = soup.find(id='tab_con1').find_all('em')[11].text.replace(',', '').replace('N/A', '0')
        per = soup.find(id='tab_con1').find_all('em')[12].text.replace(',', '').replace('N/A', '0')
        eps = soup.find(id='tab_con1').find_all('em')[13].text.replace(',', '').replace('N/A', '0')
        per_eps_date = soup.find(id='tab_con1').find_all('span', class_='date')[0].text.replace('(','').replace(')','') if len(soup.find(id='tab_con1').find_all('span', class_='date')) > 0 else ""
        estimate_per = soup.find(id='tab_con1').find_all('em')[14].text.replace(',', '').replace('N/A', '0')
        estimate_eps = soup.find(id='tab_con1').find_all('em')[15].text.replace(',', '').replace('N/A', '0')
        pbr = soup.find(id='tab_con1').find_all('em')[16].text.replace(',', '').replace('N/A', '0')
        bps = soup.find(id='tab_con1').find_all('em')[17].text.replace(',', '').replace('N/A', '0')
        pbr_bps_date = soup.find(id='tab_con1').find_all('span', class_='date')[1].text.replace('(','').replace(')','') if len(soup.find(id='tab_con1').find_all('span', class_='date')) > 1 else ""
        dvr = soup.find(id='tab_con1').find_all('em')[18].text.replace(',', '').replace('N/A', '0')                
        
        financeList = []
        financeList.append(date)
        financeList.append(str(code))
        financeList.append(totalCnt)
        financeList.append(forignerHaveLimit)
        financeList.append(forignerHaveCnt)        
        financeList.append(max52week)
        financeList.append(min52week)
        financeList.append(per)
        financeList.append(eps)
        financeList.append(per_eps_date)
        financeList.append(estimate_per)
        financeList.append(estimate_eps)
        financeList.append(pbr)
        financeList.append(bps)
        financeList.append(pbr_bps_date)
        financeList.append(dvr)    
        
        with open(financeFileName, 'a') as f:
            f.writelines(",".join([str(x) for x in financeList]) + '\n') 
        f.close()

        

In [4]:
import os
import requests 
from datetime import datetime
import pandas as pd

def crawling():
#     file_list = os.listdir('data')
#     exist_list = []
#     for file in file_list:
#         if 'finance' not in file and '.csv' in file:
#             exist_list.append(file)

    directory = 'data/'
    stockDict = GetKospi200()
    # stockDict = {'192400' : '쿠쿠홀딩스'}

    kospiListFile = "kospi_list.csv"
    bFlag = False
    if os.path.exists(directory+kospiListFile):
        bFlag = bool((pd.read_csv(directory+kospiListFile, 
                                  names=["date", "code", "name"], 
                                  header=None)["date"] == datetime.today().strftime("%Y-%m-%d")).sum())
    
    with open(directory + "/" + kospiListFile, 'a') as f:
        for key in stockDict.keys():
            print(stockDict[key], key)
            
            if not bFlag:        
                l = []
                l.append(datetime.today().strftime("%Y-%m-%d"))
                l.append(key)
                l.append(stockDict[key])
                f.writelines(",".join([str(x) for x in l]) + '\n') 

            #if str(key) not in exist_list:
            target = (key, stockDict[key])  
            GetPriceData(target) 
            getFinanceInfo(key)
    f.close()
    getKospiIndex()

In [5]:
crawling()

삼성전자 005930
SK하이닉스 000660
LG화학 051910
NAVER 035420
현대차 005380
삼성바이오로직스 207940
삼성SDI 006400
카카오 035720
셀트리온 068270
기아차 000270
현대모비스 012330
POSCO 005490
LG전자 066570
LG생활건강 051900
삼성물산 028260
SK이노베이션 096770
KB금융 105560
SK텔레콤 017670
엔씨소프트 036570
SK 034730
신한지주 055550
LG 003550
삼성생명 032830
한국전력 015760
삼성에스디에스 018260
삼성전기 009150
아모레퍼시픽 090430
하나금융지주 086790
포스코케미칼 003670
KT&G 033780
넷마블 251270
롯데케미칼 011170
S-Oil 010950
한온시스템 018880
삼성화재 000810
SK바이오팜 326030
한국조선해양 009540
LG디스플레이 034220
빅히트 352820
고려아연 010130
현대글로비스 086280
우리금융지주 316140
HMM 011200
한화솔루션 009830
KT 030200
금호석유 011780
기업은행 024110
미래에셋대우 006800
CJ제일제당 097950
현대제철 004020
한국타이어앤테크놀로지 161390
강원랜드 035250
오리온 271560
LG유플러스 032640
아모레G 002790
LG이노텍 011070
이마트 139480
코웨이 021240
한국금융지주 071050
신풍제약 019170
대한항공 003490
현대건설 000720
현대중공업지주 267250
SKC 011790
두산중공업 034020
유한양행 000100
삼성중공업 010140
한미사이언스 008930
녹십자 006280
삼성카드 029780
한진칼 180640
CJ대한통운 000120
한미약품 128940
GS 078930
롯데쇼핑 023530
두산밥캣 241560
SK케미칼 285130
롯데지주 004990
삼성증권 016360
팬오션 0

In [9]:
# 특정날짜 데이터 제거 및 reset_index
import pandas as pd
def removeData(date = "2021-03-05"):
    directory = 'data'
    file_list = os.listdir(directory)

    for file in file_list:
        if 'finance' not in file and '.csv' in file and 'kospi' not in file:   
#             print(file)
            df = pd.read_csv(directory + "/" + file)
            if "Unnamed: 0" in df.columns:
                df = pd.read_csv(directory + "/" + file).drop("Unnamed: 0", axis=1)

            if (df["date"] == date).sum() > 0:
                df = df.drop(df[df["date"] == date].index)
                df.to_csv(directory + "/" + file, index=False)

removeData("2021-03-05")

005380.csv
001680.csv
017800.csv
272210.csv
115390.csv
241590.csv
000210.csv
108670.csv
105630.csv
000990.csv
003490.csv
049770.csv
192820.csv
064350.csv
003240.csv
005180.csv
039490.csv
069260.csv
139480.csv
316140.csv
018880.csv
006260.csv
284740.csv
018260.csv
282330.csv
192400.csv
035720.csv
112610.csv
145990.csv
035250.csv
003090.csv
010950.csv
064960.csv
001120.csv
009540.csv
001450.csv
185750.csv
128940.csv
031430.csv
009150.csv
001040.csv
033780.csv
009420.csv
008930.csv
003520.csv
036570.csv
120110.csv
000270.csv
016360.csv
009240.csv
000660.csv
352820.csv
001230.csv
001740.csv
008770.csv
207940.csv
326030.csv
021240.csv
005490.csv
267250.csv
010060.csv
024110.csv
006400.csv
294870.csv
003550.csv
036460.csv
020000.csv
103140.csv
029780.csv
214320.csv
079160.csv
011790.csv
000070.csv
004000.csv
005440.csv
096770.csv
011780.csv
014820.csv
030000.csv
032830.csv
008560.csv
000100.csv
023530.csv
035420.csv
004170.csv
020560.csv
052690.csv
204320.csv
071840.csv
138930.csv
001800.csv

In [ ]:
# 특정날짜 데이터 제거 및 reset_index
import pandas as pd
def resetIndex():
#     cols = ["date", "code", "start", "close", "min", "max", "amount"]
    directory = 'data'
    file_list = os.listdir(directory)

    for file in file_list:
        if 'finance' not in file and '.csv' in file and 'kospi' not in file:     
            print(file)
            df = pd.read_csv(directory + "/" + file)
            if "Unnamed: 0" in df.columns:
                df = pd.read_csv(directory + "/" + file).drop("Unnamed: 0", axis=1)

            df = df.sort_values(by="date").reset_index().drop("index", axis=1)
            df.to_csv(directory + "/" + file, index=False)
                